In [1]:
import spacy
import random
from dateparser.search import search_dates
from rdflib import XSD
import re
from ontology.Schema import normalize_datatype

In [10]:
class NLP_Processor():
    def __init__(self,loadPath):
        self.model = spacy.load(loadPath)
        self.vector_size = len(self.model("test").vector)

    def vector(self,sentence):
        return self.model(sentence).vector

    def sentence_vector(self,sentence):
        return list(self.vector(sentence))

    def parse_sentece(self,sentence):
        sentenceAux = sentence
        
        entities = []
        #Search possibles values to strings CVs
        doc = self.model(sentence)
        for entity in doc.ents:
            entityStr = str(entity)
            sentenceAux = sentenceAux.replace(entityStr,"")
            entities.append((entityStr,entity.label_))
            
        #Search possibles values to date CVs
        print("Vai procurar datas em",sentenceAux)
        dates = search_dates(sentenceAux)
        if dates:
            for date in dates:
                entities.append((date[1],XSD.datetime))
                print("É uma data:",date[0])
                sentenceAux = sentenceAux.replace(date[0],"")
            
            
        #Search possibles values to numeric CVs
        numbers = re.findall("[-+]?\d+[\.]?\d*[eE]?[-+]?\d*", sentenceAux)
        if numbers:
            for number_str in numbers:
                sentenceAux = sentenceAux.replace(number_str,"")
                number = float(number_str)
                if number.is_integer():
                    #Number is a integer
                    normalize_datatype
                    entities.append((number,normalize_datatype(XSD.integer)))
                else:
                    #Number is a real
                    entities.append((number,normalize_datatype(XSD.float)))
        return entities,sentenceAux

In [52]:
path = "persistence/nlp/nlp_model"
nlp = NLP_Processor(path)

In [3]:
test= list(map(lambda x: x,["Quais são os medicamentos com o mesmo princípio ativo do Abacavir?",
"Quais são os medicamentos com o mesmo princípio ativo do Acetado de Atosibano?",
"Quais são os medicamentos com o mesmo princípio ativo do Gadoxetato Dissódico?",
"retorne os medicamentos com princípios ativos reopro",
"retorne os medicamentos com princípios ativos merthyolate",
"quero saber a definção de tarja",
"quero saber a definção de medicamento alotrópico",
"O que significa medicamento alotrópico",
"O que significa tarja preta",
"O que significa substância",
"Diga o que você sabe sobre o medicamento reopro",
"Diga o que você sabe sobre o medicamento buscopan",
"Diga o que você sabe sobre o medicamento bucopan composto",
"Pesquise informações sobre o medicamento buscopan",
"Pesquise informações sobre o medicamento abc",
"Quais são as apresentações do medicamento 015091909719?",
"Quais são as apresentações do medicamento 015148987892?",
"Em quais formatos eu posso comprar o medicamento 1915949489489?",
"Em quais formatos eu posso comprar o medicamento 7871981674889?",
"Em quais formatos eu posso comprar o medicamento 7815326789654?",
"Fale sobre a apresentação com o código de barras 1915949489489 em Ceará",
"Fale sobre a apresentação com o código de barras 7871981674889 em CE",
"Fale sobre a apresentação com o código de barras 7815326789654 em São Paulo",
"Você pode me dar informações sobre a apresentação com código de barras 015091909719 em Tocantins",
"Você pode me dar informações sobre a apresentação com código de barras 015148987892 em SP"
]))

random.shuffle(test)

print(test)

['Em quais formatos eu posso comprar o medicamento 7815326789654?', 'Quais são as apresentações do medicamento 015091909719?', 'Quais são as apresentações do medicamento 015148987892?', 'Diga o que você sabe sobre o medicamento buscopan', 'Pesquise informações sobre o medicamento buscopan', 'Diga o que você sabe sobre o medicamento bucopan composto', 'Em quais formatos eu posso comprar o medicamento 7871981674889?', 'Você pode me dar informações sobre a apresentação com código de barras 015148987892 em SP', 'retorne os medicamentos com princípios ativos merthyolate', 'Pesquise informações sobre o medicamento abc', 'Fale sobre a apresentação com o código de barras 7871981674889 em CE', 'Em quais formatos eu posso comprar o medicamento 1915949489489?', 'retorne os medicamentos com princípios ativos reopro', 'O que significa medicamento alotrópico', 'Quais são os medicamentos com o mesmo princípio ativo do Acetado de Atosibano?', 'quero saber a definção de tarja', 'Quais são os medicament

In [11]:
path = "persistence/nlp/nlp_model"
nlp = NLP_Processor(path)
for ele in test:
    print(ele,":")
    entities,finalString = nlp.parse_sentece(ele)
    if entities != None:
        print("Achou")
        for ent in entities:
            print("\t",ent[0],"\t-\t",ent[1])
    print("\n\n\n")

Em quais formatos eu posso comprar o medicamento 7815326789654? :
Vai procurar datas em Em quais formatos eu posso comprar o medicamento ?
Achou
	 7815326789654 	-	 http://www.linkedmed.com.br/ontology/drugs/ean@http://www.linkedmed.com.br/ontology/drugs/Apresentacao




Quais são as apresentações do medicamento 015091909719? :
Vai procurar datas em Quais são as apresentações do medicamento ?
Achou
	 015091909719 	-	 http://purl.org/dc/elements/1.1/title@http://www.linkedmed.com.br/ontology/drugs/Medicamento




Quais são as apresentações do medicamento 015148987892? :
Vai procurar datas em Quais são as apresentações do medicamento ?
Achou
	 015148987892 	-	 http://www.linkedmed.com.br/ontology/drugs/ean@http://www.linkedmed.com.br/ontology/drugs/Apresentacao




Diga o que você sabe sobre o medicamento buscopan :
Vai procurar datas em Diga o que você sabe sobre o medicamento 
Achou
	 buscopan 	-	 http://purl.org/dc/elements/1.1/title@http://www.linkedmed.com.br/ontology/drugs/Medicame

In [6]:
nlp.model.pipeline[2][1].labels

('LOC',
 'MISC',
 'ORG',
 'PER',
 'http://purl.org/dc/elements/1.1/title@http://www.linkedmed.com.br/ontology/drugs/Medicamento',
 'http://purl.org/dc/elements/1.1/title@http://www.linkedmed.com.br/ontology/drugs/PrincipioAtivo',
 'http://www.linkedmed.com.br/ontology/drugs/ean@http://www.linkedmed.com.br/ontology/drugs/Apresentacao',
 'http://www.w3.org/2000/01/rdf')

In [32]:
model = nlp.model

In [44]:
model("Você pode me dar informações sobre a apresentação com código de barras  em Tocantins").ents

()

In [67]:
nlp = NLP_Processor(path)
entities,finalString = nlp.parse_sentece("uma data 16/10/2019 e número inteiro 10 e número real 12.50 Um teste com o medicamento reopro")
if entities != None:
    for ent in entities:
        print("\t",ent[0],"\t-\t",ent[1])

	 reopro 	-	 http://purl.org/dc/elements/1.1/title@http://www.linkedmed.com.br/ontology/drugs/Medicamento
	 2019-10-16 00:00:00 	-	 http://www.w3.org/2001/XMLSchema#datetime
	 10.0 	-	 http://www.w3.org/2001/XMLSchema#integer
	 12.5 	-	 http://www.w3.org/2001/XMLSchema#decimal
